# AirBNB Paris Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib

In [ ]:
df = pd.read_csv(
    os.path.join("/", "kaggle", "input", "airbnb-listings-reviews", "Airbnb Data", "Listings.csv")
)

In [ ]:
df.isnull().sum()

too many values in district are NAN/Null, so lets drop this column

In [ ]:
df1 = df.drop(labels = ['district','host_response_time', 'host_acceptance_rate', 'host_response_rate'], axis = 1)

In [ ]:
df1.head()

In [ ]:
df1 = df1.dropna()

In [ ]:
df1.isnull().sum()

In [ ]:
df1['city'].unique()

In [ ]:
df1[df1['host_location'] == 'Turkey']['city'].unique()

In [ ]:
df1[df1['host_location'] == 'France']['city'].unique()

In [ ]:
df1[df1['host_location'] == 'Australia']['city'].unique()

there does seem to be some inconsistent data over here. Apart from Paris, none of the othe cities present in the above snippet of code are located in France. Thus we might have to ignore the 'host_location' column or we may drop it later

### Getting Paris data

Now dividing the data city wise

In [ ]:
dPar = df1[df1['city'] == 'Paris']

In [ ]:
dPar.head()

In [ ]:
dPar.describe()

In [ ]:
dPar.host_location.unique().shape

In [ ]:
dPar.neighbourhood.unique()

Sorting basis the highest number of AirBNB hosts

In [ ]:
nei_stats = dPar.groupby('neighbourhood')['neighbourhood'].agg('count').sort_values(ascending = False)
nei_stats

In [ ]:
l = []
for i in nei_stats.index:
    l.append(i)
l

In [ ]:
fig = plt.figure(figsize = (17, 5))
plt.xlabel('Districts')
plt.ylabel('No. of Hosts')
plt.title('Top districts with most BNB hosts')
plt.bar(l[:9], nei_stats[:9], width = 0.6)
plt.show()

## Average ratings of the BNBs' Distirict wise 

In [ ]:
dict = {}
for i in l:
    dict[i] = 0

for i in dPar.index:
    dict[dPar['neighbourhood'][i]] += dPar['review_scores_rating'][i]
dict

In [ ]:
k = 0
for i in dict:
    dict[i] = dict[i]/(nei_stats[k])
    k+=1
    
dict

In [ ]:
for i in dict:
    dict[i] = round(dict[i], 3)

In [ ]:
dict

In [ ]:
fig = plt.figure(figsize = (17, 5))
plt.ylim(92, 95)
plt.xlabel('District')
plt.ylabel('Average rating')
plt.title('Average ratings of the districts with most BNB hosts')
plt.bar(list(dict.keys())[:9], list(dict.values())[:9], width = 0.6, color = ['blue', 'red']*5)
plt.show()

## Top 10 districts based on Location

In [ ]:
dict1 = {}
for i in l:
    dict1[i] = 0

for i in dPar.index:
    dict1[dPar['neighbourhood'][i]] += dPar['review_scores_location'][i]
dict1

In [ ]:
k = 0
for i in dict1:
    dict1[i] = dict1[i]/(nei_stats[k])
    k+=1
    
dict1

In [ ]:
sorted_dict = {}
sorted_keys = sorted(dict1, key = dict1.get)
sorted_keys

In [ ]:
for j in sorted_keys:
    sorted_dict[j] = dict1[j]
sorted_dict

In [ ]:
fig = plt.figure(figsize = (17, 5))
plt.ylim(9.1, 10)
plt.xlabel('Districts')
plt.ylabel('Average Location Rating')
plt.title('Top 10 districts based on location')
plt.bar(list(sorted_dict.keys())[-10:], list(sorted_dict.values())[-10:], width = 0.6, color = ['green', 'orange']*5)
plt.show()

## Types of BNB hosts

In [ ]:
dPar.describe()

In [ ]:
dPar['room_type'].unique()

In [ ]:
room_stats = dPar.groupby('room_type')['room_type'].agg('count').sort_values(ascending = False)

In [ ]:
room_stats

In [ ]:
l = [i for i in room_stats.index]
l

In [ ]:
def func(x, room_stats):
    #pct = (x*100)/room_stats.sum()
    return "{:.1f}%".format(x)
    
fig = plt.figure(figsize = (8, 8))
plt.pie(room_stats,
        labels = l,
        autopct = lambda pct: func(pct, room_stats),
        explode = [0.2, 0.1, 0.1, 0.2],
        shadow = True)
plt.title('Types of Hosts')
plt.show()

## Factor importance of amenities for a good rating

there might be different amenities available at different BNB host locations. So lets see just the importance of 10 most commonly
found amenities

In [ ]:
dPar.head()

In [ ]:
amen = dPar['amenities'].unique()
amen

converting the string value to list

In [ ]:
import ast
def Lis(x):
    arr = ast.literal_eval(x)
    return arr

Lis("['Hello']")

dPar['amen_as_list'] = dPar['amenities'].apply(Lis)

In [ ]:
dPar['amen_as_list']

Amenities most commonly found at Hosts' locations

In [ ]:
amenities = {}

for i in dPar['amen_as_list'].index:
    for j in range(len(dPar['amen_as_list'][i])):
        if(dPar['amen_as_list'][i][j] not in amenities):
            amenities[(dPar['amen_as_list'][i][j])] = 1
        else:
            amenities[(dPar['amen_as_list'][i][j])] += 1
amenities

In [ ]:
#new_dict = [(key, value) for value, key in amenities.items()]
#dict(new_dict)
new_amen = sorted(amenities.items(), key = lambda x: x[1])

In [ ]:
new_amen[-25:]

In [ ]:
newDict = []
for i in range(15,25):
    newDict.append(new_amen[-25:][i][0])
newDict

In [ ]:
dnew = pd.DataFrame()
dnew['scores'] = dPar['review_scores_rating']
dnew

In [ ]:
for i in newDict:
    dnew[i] = 0

dnew

Filling the new data of amentities

In [ ]:
for i in dPar['amen_as_list'].index:
    for j in range(len(dPar['amen_as_list'][i])):
        if dPar['amen_as_list'][i][j] in newDict:
            dnew[dPar['amen_as_list'][i][j]][i] = 1

dnew

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etf = ExtraTreesClassifier()

X = dnew.drop(labels = ['scores'], axis = 1)
y = dnew['scores']

etf.fit(X, y)

In [ ]:
feature_importance = etf.feature_importances_
feature_importance

In [ ]:
for i in range(10):
    feature_importance[i] = feature_importance[i]*100

In [ ]:
newDict[4] = 'Long term stays'

In [ ]:
fig = plt.figure(figsize=(17, 5))
plt.ylim(6,12)
plt.bar(newDict, feature_importance)
plt.xlabel('Amenities')
plt.ylabel('Contributing factor to a good score')
plt.title('Ameninties')
plt.show()